<a href="https://colab.research.google.com/github/ckrstars/CGS2/blob/main/suvidha_foundation_ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# FINAL FOOLPROOF SCRIPT - V3
# This script does NOT use pyrouge and relies on modern libraries.
# ==============================================================================

# ------------------------------------------------------------------------------
# Part 1: Environment Setup
# Install all necessary libraries and clone the code repository.
# ------------------------------------------------------------------------------
print("--- Part 1: Setting up the Environment ---")
# Install datasets, a compatible fsspec, and the MODERN rouge_score library
!pip install datasets "fsspec==2023.6.0" rouge_score nltk > /dev/null
print("Libraries installed.")

# Clone the required GitHub repository
!git clone https://github.com/atulkum/pointer_summarizer.git
print("Repository cloned.")

# CRITICAL STEP: Change the working directory. All subsequent commands
# will run from inside this folder, preventing "No such file" errors.
%cd /content/pointer_summarizer
print("Changed working directory to /content/pointer_summarizer/")


# ------------------------------------------------------------------------------
# Part 2: Data Preparation
# Download CNN/DailyMail and format it into the .story files the repo needs.
# ------------------------------------------------------------------------------
print("\n--- Part 2: Downloading and Preparing Data ---")
import os
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Define the directory structure required by the training script
data_dir = '/content/pointer_summarizer/data/cnn_dm'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

def write_to_text_files(dataset_split, directory, split_name):
    """Converts Hugging Face dataset splits to .story files."""
    print(f"Processing '{split_name}' split...")
    # Use a smaller subset for faster setup. For final results, use the full dataset.
    num_examples = 10000 if split_name == 'train' else 1000

    for i, example in enumerate(dataset_split.select(range(num_examples))):
        if i > 0 and i % 2000 == 0:
            print(f"  ...processed {i} examples...")

        article_text = example['article']
        highlight_text = example['highlights']

        formatted_highlight = "\n@highlight\n\n" + highlight_text
        file_content = article_text + formatted_highlight

        file_path = os.path.join(directory, f"{i:06d}.story")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(file_content)

    print(f"Finished '{split_name}' split with {num_examples} files.")

# Run the conversion
write_to_text_files(dataset['train'], train_dir, 'train')
write_to_text_files(dataset['validation'], val_dir, 'val')
write_to_text_files(dataset['test'], test_dir, 'test')
print("Data preparation complete.")


# ------------------------------------------------------------------------------
# Part 3: Patching the Code
# Automatically fix the repository's code to be compatible with Python 3.
# ------------------------------------------------------------------------------
print("\n--- Part 3: Patching Code for Python 3 Compatibility ---")

# Fix 1: Change Python 2's 'Queue' to Python 3's 'queue'
!sed -i 's/import Queue/import queue/g' data_util/batcher.py
!sed -i 's/Queue.Queue/queue.Queue/g' data_util/batcher.py
print("✅ Fixed 'Queue' import.")

# Fix 2: Change all implicit relative imports to explicit relative imports
!sed -i 's/from model import Model/from .model import Model/g' training_ptr_gen/train.py
!sed -i 's/from training_ptr_gen.train_util/from .train_util/g' training_ptr_gen/train.py
!sed -i 's/from training_ptr_gen.attention_decoder/from .attention_decoder/g' training_ptr_gen/model.py
!sed -i 's/from training_ptr_gen.beam_search/from .beam_search/g' training_ptr_gen/model.py
!sed -i 's/from model import Model/from .model import Model/g' training_ptr_gen/decode.py
!sed -i 's/from training_ptr_gen.beam_search/from .beam_search/g' training_ptr_gen/decode.py
print("✅ Fixed relative imports.")
print("Code patching complete.")

# ------------------------------------------------------------------------------
# Part 4: Train the Model
# Run the training script with all the correct parameters.
# ------------------------------------------------------------------------------
print("\n--- Part 4: Starting Model Training ---")
print("This is the final step. It will take several hours. Please be patient.")

# The final, correct command to run the training
!python -m training_ptr_gen.train \
    --is_pointer_gen \
    --data_path='data/cnn_dm' \
    --vocab_path='data/cnn_dm/vocab' \
    --exp_name='pgn_final_run' \
    --batch_size=8 \
    --epochs=15 \
    --vocab_size=50000 \
    --log_root='log'

print("\n--- Training script has successfully started! Monitor the loss values. ---")

--- Part 1: Setting up the Environment ---
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nv

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Processing 'train' split...
  ...processed 2000 examples...
  ...processed 4000 examples...
  ...processed 6000 examples...
  ...processed 8000 examples...
Finished 'train' split with 10000 files.
Processing 'val' split...
Finished 'val' split with 1000 files.
Processing 'test' split...
Finished 'test' split with 1000 files.
Data preparation complete.

--- Part 3: Patching Code for Python 3 Compatibility ---
✅ Fixed 'Queue' import.
✅ Fixed relative imports.
Code patching complete.

--- Part 4: Starting Model Training ---
This is the final step. It will take several hours. Please be patient.
2025-07-01 13:53:29.675468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751378009.693987     918 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:17513

In [2]:
# We must be in the correct directory first.
%cd /content/pointer_summarizer

# This command finds "import config" in the batcher.py file
# and replaces it with the correct "from . import config"
!sed -i 's/import config/from . import config/g' data_util/batcher.py

print("✅ Final patch applied to data_util/batcher.py")

/content/pointer_summarizer
✅ Final patch applied to data_util/batcher.py


In [3]:
# This command will print the line containing our fix from batcher.py
!grep "from . import config" data_util/batcher.py

from . import config


In [4]:
# We are already in the /content/pointer_summarizer directory.
# Now, we can finally run the training command successfully.

!python -m training_ptr_gen.train \
    --is_pointer_gen \
    --data_path='data/cnn_dm' \
    --vocab_path='data/cnn_dm/vocab' \
    --exp_name='pgn_final_run' \
    --batch_size=8 \
    --epochs=15 \
    --vocab_size=50000 \
    --log_root='log'

2025-07-01 13:57:11.200450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751378231.220242    2299 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751378231.225820    2299 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 13:57:11.244859: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>",

In [5]:
# Ensure we are in the main project directory
%cd /content/pointer_summarizer

# This command finds all Python 2 print statements and converts them to Python 3 style
# It looks for 'print ' followed by something and wraps the "something" in parentheses.
# We apply it to the files we know have this issue.
!sed -i "s/print '\(.*\)'/print('\1')/g" data_util/data.py
!sed -i "s/print u'\(.*\)'/print(u'\1')/g" training_ptr_gen/train_util.py
!sed -i "s/print 'running eval on ...'/print('running eval on ...')/g" training_ptr_gen/train_util.py # A specific case
!sed -i "s/print 'Finished decoding'/print('Finished decoding')/g" training_ptr_gen/decode.py

print("✅ Fixed all known Python 2 'print' statements.")

/content/pointer_summarizer
✅ Fixed all known Python 2 'print' statements.


In [6]:
# Check the line in data_util/data.py that previously caused the error
!grep "Warning: incorrectly formatted line" data_util/data.py

          print('Warning: incorrectly formatted line in vocabulary file: %s\n') % line


In [9]:
# We are in the correct directory. All patches are applied.
# This command will now run without syntax errors.

!python -m training_ptr_gen.train \
    --is_pointer_gen \
    --data_path='data/cnn_dm' \
    --vocab_path='data/cnn_dm/vocab' \
    --exp_name='pgn_final_run' \
    --batch_size=8 \
    --epochs=15 \
    --vocab_size=50000 \
    --log_root='log'

2025-07-01 14:02:02.157002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751378522.176897    4097 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751378522.182580    4097 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 14:02:02.201588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>",

In [10]:
# This command will print the entire content of the file we just edited.
# It should match the code block above exactly.
%cd /content/pointer_summarizer
!cat data_util/utils.py

/content/pointer_summarizer
# This is the new, complete content for /content/pointer_summarizer/data_util/utils.py

import os
import time

def calc_running_avg_loss(loss, running_avg_loss, step, decay=0.99):
  """Calculates the running average of a loss value."""
  if running_avg_loss == 0:
    running_avg_loss = loss
  else:
    running_avg_loss = running_avg_loss * decay + (1 - decay) * loss
  running_avg_loss = min(running_avg_loss, 12) # clip
  return running_avg_loss

#
# All other functions related to pyrouge have been removed from this file
# to eliminate the dependency.
#

In [11]:
# We are in the correct directory. All patches, including the final one, are applied.
# This command will now succeed.

!python -m training_ptr_gen.train \
    --is_pointer_gen \
    --data_path='data/cnn_dm' \
    --vocab_path='data/cnn_dm/vocab' \
    --exp_name='pgn_final_run' \
    --batch_size=8 \
    --epochs=15 \
    --vocab_size=50000 \
    --log_root='log'

2025-07-01 14:09:05.717670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751378945.736668    6661 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751378945.742268    6661 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 14:09:05.761555: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>",

In [12]:
# We are in the correct directory. All patches are applied.

!python -m training_ptr_gen.train \
    --is_pointer_gen \
    --data_path='data/cnn_dm' \
    --vocab_path='data/cnn_dm/vocab' \
    --exp_name='pgn_final_run' \
    --batch_size=8 \
    --epochs=15 \
    --vocab_size=50000 \
    --log_root='log'

2025-07-01 14:11:10.311787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751379070.353382    7430 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751379070.363712    7430 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 14:11:10.393662: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
usage: train.py [-h] [-m MODEL_FILE_PATH]
train.py: error: unrecognized arguments: --is_pointer_gen --data_path=data/

In [13]:
# We are in the correct directory. All patches and configurations are applied.
# This command has NO arguments.

!python -m training_ptr_gen.train

2025-07-01 14:14:12.075603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751379252.094639    8531 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751379252.100965    8531 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 14:14:12.122389: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>",

In [14]:
import os
import collections
from tensorflow.core.example import example_pb2

# --- CONFIGURATION (Must match your data_util/config.py) ---
VOCAB_SIZE = 50000
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'
PAD_TOKEN = '[PAD]'
UNKNOWN_TOKEN = '[UNK]'
START_DECODING = '[START]'
STOP_DECODING = '[STOP]'

# The directory where your .story files for training are located
train_data_dir = "/content/pointer_summarizer/data/cnn_dm/train"
# The exact path where the vocab file needs to be saved
vocab_file_path = "/content/pointer_summarizer/data/cnn_dm/vocab"

print("Starting vocabulary creation...")
print(f"Reading files from: {train_data_dir}")

# --- SCRIPT LOGIC ---
vocab_counter = collections.Counter()

# List all the .story files in the training directory
filelist = os.listdir(train_data_dir)
print(f"Found {len(filelist)} story files.")

for i, filename in enumerate(filelist):
    if i % 1000 == 0:
        print(f"  Processing file {i} of {len(filelist)}...")

    filepath = os.path.join(train_data_dir, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        # Read the file content and split into lines
        lines = f.read().split('\n')

        # The first part is the article, before the @highlight
        article_lines = []
        for line in lines:
            if line.startswith('@highlight'):
                break
            article_lines.append(line)

        # Tokenize the article text
        article_text = ' '.join(article_lines)
        tokens = article_text.split()

        # Add words to the counter
        for token in tokens:
            if token.strip(): # Ensure it's not an empty string
                vocab_counter.update([token])

print("\nFinished counting words.")

# Create the vocabulary file
print(f"Writing vocabulary file to: {vocab_file_path}")
with open(vocab_file_path, 'w', encoding='utf-8') as writer:
    # Get the most common words, leaving space for special tokens
    most_common = vocab_counter.most_common(VOCAB_SIZE - 4)

    # Write special tokens first
    writer.write(UNKNOWN_TOKEN + '\n')
    writer.write(PAD_TOKEN + '\n')
    writer.write(START_DECODING + '\n')
    writer.write(STOP_DECODING + '\n')

    # Write the most common words
    for word, count in most_common:
        writer.write(word + '\n')

print("\n✅ Vocabulary file created successfully!")

Starting vocabulary creation...
Reading files from: /content/pointer_summarizer/data/cnn_dm/train
Found 10000 story files.
  Processing file 0 of 10000...
  Processing file 1000 of 10000...
  Processing file 2000 of 10000...
  Processing file 3000 of 10000...
  Processing file 4000 of 10000...
  Processing file 5000 of 10000...
  Processing file 6000 of 10000...
  Processing file 7000 of 10000...
  Processing file 8000 of 10000...
  Processing file 9000 of 10000...

Finished counting words.
Writing vocabulary file to: /content/pointer_summarizer/data/cnn_dm/vocab

✅ Vocabulary file created successfully!


In [15]:
# This command should now list the 'vocab' file
!ls -l /content/pointer_summarizer/data/cnn_dm/

total 764
drwxr-xr-x 2 root root  36864 Jul  1 13:53 test
drwxr-xr-x 2 root root 274432 Jul  1 13:53 train
drwxr-xr-x 2 root root  36864 Jul  1 13:53 val
-rw-r--r-- 1 root root 418077 Jul  1 14:16 vocab


In [16]:
# We are in the correct directory. All patches are applied. The vocab file now exists.

!python -m training_ptr_gen.train

2025-07-01 14:16:39.396484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751379399.414868    9417 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751379399.420441    9417 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/pointer_summarizer/training_ptr_gen/train.py", line 149, in <module>
    train_processor = Train()
                      ^^^^^^^
  File "/content/pointer_summarizer/training_ptr_gen/train.py", line 24, in __init__
    self.vocab = Vocab(config.vocab_path, config.vocab_size)
    

In [17]:
# We are in the correct directory. All files exist and all code is patched.

!python -m training_ptr_gen.train

Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [18]:
# In a new cell, run this improved vocab builder.
import os
import collections

# --- CONFIGURATION ---
VOCAB_SIZE = 50000
train_data_dir = "/content/pointer_summarizer/data/cnn_dm/train"
vocab_file_path = "/content/pointer_summarizer/data/cnn_dm/vocab"

print("Starting CORRECTED vocabulary creation...")

vocab_counter = collections.Counter()

filelist = os.listdir(train_data_dir)
print(f"Found {len(filelist)} story files.")

for i, filename in enumerate(filelist):
    if i > 0 and i % 1000 == 0:
        print(f"  Processing file {i}...")

    filepath = os.path.join(train_data_dir, filename)
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            file_content = f.read()
            # The article is everything before the first @highlight
            article_content = file_content.split('@highlight')[0]
            tokens = article_content.strip().split()
            vocab_counter.update(tokens)
    except Exception as e:
        print(f"Could not read file {filename}: {e}")

print(f"\nFinished counting. Total unique words: {len(vocab_counter)}")

# Create the vocabulary file
with open(vocab_file_path, 'w', encoding='utf-8') as writer:
    # Get the most common words
    most_common = vocab_counter.most_common(VOCAB_SIZE)
    for word, count in most_common:
        writer.write(word + '\n')

print(f"\n✅ Vocabulary file with ~{VOCAB_SIZE} words created successfully at: {vocab_file_path}")

Starting CORRECTED vocabulary creation...
Found 10000 story files.
  Processing file 1000...
  Processing file 2000...
  Processing file 3000...
  Processing file 4000...
  Processing file 5000...
  Processing file 6000...
  Processing file 7000...
  Processing file 8000...
  Processing file 9000...

Finished counting. Total unique words: 231879

✅ Vocabulary file with ~50000 words created successfully at: /content/pointer_summarizer/data/cnn_dm/vocab


In [19]:
# We are in the correct directory. All files and configs are now correct.

!python -m training_ptr_gen.train

Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [20]:
# We are in the correct directory. All files, configs, and code are now fully Python 3 compatible.

!python -m training_ptr_gen.train

Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [21]:
# This command will now work.

!python -m training_ptr_gen.train

Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [22]:
!python /content/pointer_summarizer/process_stories.py


2025-07-01 15:10:40.494096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751382640.512235   28829 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751382640.517833   28829 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Writing story 0 of 10000; 0.00 percent done
Writing story 1000 of 10000; 10.00 percent done
Writing story 2000 of 10000; 20.00 percent done
Writing story 3000 of 10000; 30.00 percent done
Writing story 4000 of 10000; 40.00 percent done
Writing story 5000 of 10000; 50.00 percent done
Writing story 6000 of 10000; 60.00 percent done
Writing story 7000 of 10000; 70.00 percent done
Writing story 8000 of 10000; 80.00 percent done
Writing s

In [23]:
# This is the final step. Run this in a new cell.

!python -m training_ptr_gen.train

2025-07-01 15:12:45.961243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751382765.979173   29597 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751382765.984641   29597 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: passenger.
Exception in thread Thread-1 (fill_example_queue):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **se